Extract: Players_Active

In [11]:
import requests
import pandas as pd
from config import api_key

def fetch_and_inspect_api_data():
    url = f'https://api.sportsdata.io/v3/nhl/scores/json/Players?key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()  # Convert the response to JSON
        
        # Convert the data to a pandas DataFrame for inspection
        df = pd.DataFrame(data)
        
        # Inspect the first few rows of the DataFrame
        print(df.head())
        
        return df
    else:
        print("Failed to retrieve data from the API")
        return None

# Now, call the function to fetch the data and inspect it
players_active_df = fetch_and_inspect_api_data()


   PlayerID FirstName    LastName           Status  TeamID Team Position  \
0  30000007     Carey       Price  Injured Reserve       4  MON        G   
1  30000012      Lars       Eller           Active      14  PIT        C   
2  30000015   Brendan   Gallagher           Active       4  MON       RW   
3  30000019       Max  Pacioretty           Active      15  WAS       LW   
4  30000031     Peter     Holland           Minors      19  COL        C   

   Jersey Catches Shoots  ...  DepthChartPosition  DepthChartOrder  \
0    31.0       L   None  ...                None              NaN   
1    20.0    None      L  ...                   C              1.0   
2    11.0    None      R  ...                  LW              1.0   
3    67.0    None      L  ...                  LW              1.0   
4     NaN    None      L  ...                None              NaN   

  GlobalTeamID   FantasyDraftName FantasyDraftPlayerID UsaTodayPlayerID  \
0     30000004               None              

Transform: Players_active


In [12]:
def transform_players_active_df_v2(df):
    # Step 1: Select only the relevant columns. Adjust this list as per your requirement.
    relevant_columns = ['PlayerID', 'FirstName', 'LastName', 'Status', 'TeamID', 'Team', 'Position', 'Height', 'Weight', 'BirthDate', 'BirthCity', 'BirthState']
    df = df[relevant_columns]

    # Step 2: Drop duplicates based on 'PlayerID'
    df.drop_duplicates(subset='PlayerID', inplace=True)

    # Step 3: Combine 'BirthCity' and 'BirthState' into 'BirthPlace'
    # Use a lambda function to handle rows where either city or state might be missing
    df['BirthPlace'] = df.apply(lambda row: ','.join(filter(None, [row['BirthCity'], row['BirthState']])), axis=1)
    
    # Format BirthDate to mm/dd/yyyy
    df['BirthDate'] = pd.to_datetime(df['BirthDate']).dt.strftime('%m/%d/%Y')

    # Drop the original 'BirthCity' and 'BirthState' columns as they're no longer needed
    df.drop(['BirthCity', 'BirthState'], axis=1, inplace=True)

    # Inspect the transformed DataFrame 
    print(df.head())
    print(df.info())

    return df

# Apply the transformation function
players_active_df_transformed = transform_players_active_df_v2(players_active_df)


   PlayerID FirstName    LastName           Status  TeamID Team Position  \
0  30000007     Carey       Price  Injured Reserve       4  MON        G   
1  30000012      Lars       Eller           Active      14  PIT        C   
2  30000015   Brendan   Gallagher           Active       4  MON       RW   
3  30000019       Max  Pacioretty           Active      15  WAS       LW   
4  30000031     Peter     Holland           Minors      19  COL        C   

   Height  Weight   BirthDate      BirthPlace  
0      75     217  08/16/1987  Anahim Lake,BC  
1      74     205  05/08/1989         Rodovre  
2      69     183  05/06/1992     Edmonton,AB  
3      74     217  11/20/1988   New Canaan,CT  
4      74     193  01/14/1991      Toronto,ON  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1855 entries, 0 to 1854
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PlayerID    1855 non-null   int64 
 1   FirstName   1855 non-nu

C:\Users\lenar\AppData\Local\Temp\ipykernel_9712\2405883503.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset='PlayerID', inplace=True)
C:\Users\lenar\AppData\Local\Temp\ipykernel_9712\2405883503.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BirthPlace'] = df.apply(lambda row: ','.join(filter(None, [row['BirthCity'], row['BirthState']])), axis=1)
C:\Users\lenar\AppData\Local\Temp\ipykernel_9712\2405883503.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [13]:
# Check data types

players_active_df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1855 entries, 0 to 1854
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PlayerID    1855 non-null   int64 
 1   FirstName   1855 non-null   object
 2   LastName    1855 non-null   object
 3   Status      1855 non-null   object
 4   TeamID      1855 non-null   int64 
 5   Team        1855 non-null   object
 6   Position    1855 non-null   object
 7   Height      1855 non-null   int64 
 8   Weight      1855 non-null   int64 
 9   BirthDate   1854 non-null   object
 10  BirthPlace  1855 non-null   object
dtypes: int64(4), object(7)
memory usage: 159.5+ KB


In [14]:
# Establish database connection

import psycopg2

def create_connection():
    try:
        conn = psycopg2.connect(
            dbname='NHL',
            user='postgres',
            password='postgres',
            host='localhost'

        )
        print("Successfully connected to the database.")
        return conn
    except psycopg2.Error as e:
        print(f'Error: Could not make connection to PostGreSQL database')
        print(e)

conn = create_connection()

Successfully connected to the database.


Load: Players_Active

In [15]:
from psycopg2.extras import execute_batch

def load_data_to_db(conn, df, table_name):
    """
    Load data from a DataFrame to a specified table in the PostgreSQL database.
    
    :param conn: Connection object to the database
    :param df: DataFrame containing the data to load
    :param table_name: Name of the table where data will be inserted
    """
    cursor = conn.cursor()
    
    # Define INSERT INTO statement
    columns = df.columns.tolist()  # Get the column names from the DataFrame
    columns_str = ', '.join(columns)  # Create a string of column names for the SQL statement
    values_str = ', '.join(['%s' for _ in columns])  # Create a string of '%s' placeholders for the values
    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str}) ON CONFLICT DO NOTHING;"
    
    # Prepare the data for insertion
    data_to_insert = [tuple(row) for row in df.to_numpy()]
    
    try:
        # Use execute_batch for efficient bulk inserts
        execute_batch(cursor, insert_query, data_to_insert)
        conn.commit()
        print("Data loaded successfully into the database.")
    except psycopg2.Error as e:
        conn.rollback()  # Rollback the transaction in case of error
        print("An error occurred while inserting data into the database:")
        print(e)
    finally:
        cursor.close()

table_name = 'players'  
load_data_to_db(conn, players_active_df_transformed, table_name)



Data loaded successfully into the database.


Extract: Teams

In [16]:
import requests
import pandas as pd
from config import api_key

def fetch_and_inspect_api_data():
    url = f'https://api.sportsdata.io/v3/nhl/scores/json/AllTeams?key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()  # Convert the response to JSON
        
        # Convert the data to a pandas DataFrame for inspection
        df = pd.DataFrame(data)
        
        # Inspect the first few rows of the DataFrame
        print(df.head())
        
        return df
    else:
        print("Failed to retrieve data from the API")
        return None

# Now, call the function to fetch the data and inspect it
teams_active_df = fetch_and_inspect_api_data()


   TeamID  Key  Active      City       Name  StadiumID Conference  Division  \
0       1  BOS    True    Boston     Bruins        3.0    Eastern  Atlantic   
1       2  BUF    True   Buffalo     Sabres        4.0    Eastern  Atlantic   
2       3  DET    True   Detroit  Red Wings       11.0    Eastern  Atlantic   
3       4  MON    True  Montreal  Canadiens       16.0    Eastern  Atlantic   
4       5  OTT    True    Ottawa   Senators       21.0    Eastern  Atlantic   

  PrimaryColor SecondaryColor TertiaryColor QuaternaryColor  \
0       000000         FDB717          None            None   
1       003087         FFB81C        FFFFFF            None   
2       C8102E         FFFFFF          None            None   
3       A6192E         FFFFFF        001E62            None   
4       010101         C8102E        B9975B          FFFFFF   

                                    WikipediaLogoUrl WikipediaWordMarkUrl  \
0  https://upload.wikimedia.org/wikipedia/commons...                 

Transform: Teams

In [17]:
teams_active_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TeamID                40 non-null     int64  
 1   Key                   40 non-null     object 
 2   Active                40 non-null     bool   
 3   City                  40 non-null     object 
 4   Name                  40 non-null     object 
 5   StadiumID             32 non-null     float64
 6   Conference            32 non-null     object 
 7   Division              32 non-null     object 
 8   PrimaryColor          32 non-null     object 
 9   SecondaryColor        32 non-null     object 
 10  TertiaryColor         29 non-null     object 
 11  QuaternaryColor       13 non-null     object 
 12  WikipediaLogoUrl      32 non-null     object 
 13  WikipediaWordMarkUrl  0 non-null      object 
 14  GlobalTeamID          40 non-null     int64  
 15  HeadCoach             32 

In [18]:
def transform_teams_active_df_v2(df):
    
    # Make a 'Colors' column
    df['Colors'] = df.apply(lambda row: ','.join(filter(None, [row['PrimaryColor'], row['SecondaryColor'], row['TertiaryColor'], row['QuaternaryColor']])), axis=1)
    df.drop(['PrimaryColor', 'SecondaryColor', 'TertiaryColor', 'QuaternaryColor'], axis=1, inplace=True)

    # Drop duplicates 
    df.drop_duplicates(subset='TeamID', inplace=True)

    # Reorder columns
    df = df[['TeamID', 'Name', 'Key', 'City', 'StadiumID', 'Conference', 'Division', 'HeadCoach', 'Colors',  'GlobalTeamID', 'Active']]

    # Rename column
    df.rename(columns={'Key': 'Abbreviation'}, inplace=True)
    df.rename(columns={'Colors': 'ColorCode'}, inplace=True)
    df.rename(columns={'Name': 'Name_'}, inplace=True)

    
    # Format BirthDate to mm/dd/yyyy
    #df['BirthDate'] = pd.to_datetime(df['BirthDate']).dt.strftime('%m/%d/%Y')

    
    # Inspect the transformed DataFrame 
    #print(df.head())
    print(df.info())

    return df

# Apply the transformation function
teams_active_df_transformed = transform_teams_active_df_v2(teams_active_df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TeamID        40 non-null     int64  
 1   Name_         40 non-null     object 
 2   Abbreviation  40 non-null     object 
 3   City          40 non-null     object 
 4   StadiumID     32 non-null     float64
 5   Conference    32 non-null     object 
 6   Division      32 non-null     object 
 7   HeadCoach     32 non-null     object 
 8   ColorCode     40 non-null     object 
 9   GlobalTeamID  40 non-null     int64  
 10  Active        40 non-null     bool   
dtypes: bool(1), float64(1), int64(2), object(7)
memory usage: 3.3+ KB
None


C:\Users\lenar\AppData\Local\Temp\ipykernel_9712\3641362884.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Key': 'Abbreviation'}, inplace=True)
C:\Users\lenar\AppData\Local\Temp\ipykernel_9712\3641362884.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Colors': 'ColorCode'}, inplace=True)
C:\Users\lenar\AppData\Local\Temp\ipykernel_9712\3641362884.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [19]:
# Establish database connection

import psycopg2

def create_connection():
    try:
        conn = psycopg2.connect(
            dbname='NHL',
            user='postgres',
            password='postgres',
            host='localhost'

        )
        print("Successfully connected to the database.")
        return conn
    except psycopg2.Error as e:
        print(f'Error: Could not make connection to PostGreSQL database')
        print(e)

conn = create_connection()

Successfully connected to the database.


In [20]:
from psycopg2.extras import execute_batch

def load_data_to_db(conn, df, table_name):
    """
    Load data from a DataFrame to a specified table in the PostgreSQL database.
    
    :param conn: Connection object to the database
    :param df: DataFrame containing the data to load
    :param table_name: Name of the table where data will be inserted
    """
    cursor = conn.cursor()
    
    # Define INSERT INTO statement
    columns = df.columns.tolist()  # Get the column names from the DataFrame
    columns_str = ', '.join(columns)  # Create a string of column names for the SQL statement
    values_str = ', '.join(['%s' for _ in columns])  # Create a string of '%s' placeholders for the values
    insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({values_str}) ON CONFLICT DO NOTHING;"
    
    # Prepare the data for insertion
    data_to_insert = [tuple(row) for row in df.to_numpy()]
    
    try:
        # Use execute_batch for efficient bulk inserts
        execute_batch(cursor, insert_query, data_to_insert)
        conn.commit()
        print("Data loaded successfully into the database.")
    except psycopg2.Error as e:
        conn.rollback()  # Rollback the transaction in case of error
        print("An error occurred while inserting data into the database:")
        print(e)
    finally:
        cursor.close()

table_name = "teams_active"  
load_data_to_db(conn, teams_active_df_transformed, table_name)



Data loaded successfully into the database.
